In [ ]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand 

GNU General Public License v3.0


'''


In [ ]:
# Standard Imports
%matplotlib inline
%load_ext autoreload
%autoreload
import matplotlib
import scipy
import matplotlib.pyplot as plt
import datetime
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
import os
import re
import glob
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import dask
# dask.set_options(get=dask.threaded.get)

from dask.distributed import Client
client = Client()

# ESIO Imports

from esio import EsioData as ed

In [ ]:
client

In [ ]:
# General plotting settings
sns.set_style('whitegrid')
sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})

In [ ]:
E = ed.EsioData.load()
model_dir = E.model_dir
# Directories
# Define models to plot
all_models = list(E.model.keys())
all_models = [x for x in all_models if x!='piomas'] # remove some models
# all_models = ['cma']
runType='forecast'
updateall = False
data_vars = ['sic']

In [ ]:
ds_region = xr.open_mfdataset(os.path.join(E.grid_dir, 'sio_2016_mask_Update.nc'))

In [ ]:
# output dir
data_out = os.path.join(model_dir, 'MME', runType, 'sipn_nc')
if not os.path.exists(data_out):
    os.makedirs(data_out)

In [ ]:
# First loop through all models and lazily open them
mod_l = []
cen = 0 # Current ensemble number
for model in all_models:
    if model=='MME':
        continue
    print(model)
    
    data_dir = E.model[model][runType]['sipn_nc']
    all_files = glob.glob(os.path.join(data_dir, '*.nc'))
    
    # Check for any files
    if not all_files:
        continue
        
    # Remove any "empty" files (sometimes happends with ecmwf downloads)
    all_files_new = []
    for cf in all_files:
        if os.stat(cf).st_size > 0:
            all_files_new.append(cf)
    all_files = sorted(all_files_new) # Replace and sort


    ds = xr.open_mfdataset(all_files , concat_dim='init_time', 
                           chunks={'fore_time':10, 'ensemble': 5, 'init_time': 10, 'nj': 304, 'ni': 448},
                          autoclose=True,
                          parallel=True) 

    # Update ensemble number
    ds.coords['ensemble'] = np.arange(cen,cen+ds.ensemble.size)
    cen = cen + ds.ensemble.size
    
    # Grab stuff
    ds = ds[data_vars]
    
    # Drop stuff
    if 'valid_time' in ds.coords:
        ds = ds.drop(['valid_time'])
        
    print(ds.init_time.values[0], ds.init_time.values[-1])
            
    ds.coords['model'] = model
    mod_l.append(ds)

In [ ]:
# Concat data by model
ds_all = xr.concat(mod_l, dim='ensemble')

In [ ]:
ds_all

In [ ]:
# Simple multi-model mean
ds_mme = ds_all.mean(dim='ensemble')

In [ ]:
# Expand dims
ds_mme = import_data.expand_to_sipn_dims(ds_mme)

In [ ]:
# Compute
ds_mme.compute()

In [ ]:
# Write out
ds_mme.to_netcdf(os.path.join(data_out, 'MME.nc'))